## Old/Traditional Approach

In [7]:
import os
from dotenv import load_dotenv
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from langchain_groq import ChatGroq
from langchain.schema import BaseRetriever, Document

# Load environment variables
load_dotenv()
GROQ_API_KEY = os.getenv("GROQ_API_KEY")
if not GROQ_API_KEY:
    raise ValueError("Missing GROQ_API_KEY in .env file")

# Paths and model setup
ROOT_DIR = os.path.abspath(os.path.join(os.getcwd(), "..")) \
    if os.path.basename(os.getcwd()) == "notebooks" else os.getcwd()
CHROMA_DIR = os.path.join(ROOT_DIR, "chroma_db")

EMBED_MODEL = "BAAI/bge-large-en-v1.5"
MODEL_NAME = "llama-3.3-70b-versatile"

print(f"Using Chroma DB from: {CHROMA_DIR}")

# Load vector store and embeddings
embeddings = HuggingFaceEmbeddings(model_name=EMBED_MODEL)
db = Chroma(persist_directory=CHROMA_DIR, embedding_function=embeddings)
retriever = db.as_retriever(search_type="similarity", search_kwargs={"k": 5})

# Initialize LLM
llm = ChatGroq(
    api_key=GROQ_API_KEY,
    model=MODEL_NAME,
    temperature=0.2,
    max_tokens=512,
)

# Prompt template
prompt = PromptTemplate(
    template="""
You are a helpful legal assistant. Use the context below to answer clearly and accurately.

Context:
{context}

Question:
{question}

Answer:
""",
    input_variables=["context", "question"],
)

# Metadata-aware retriever
class MetadataAwareRetriever(BaseRetriever):
    model_config = {"extra": "allow"}  # allow custom attributes

    def __init__(self, retriever, priority_keywords=None, **kwargs):
        super().__init__(**kwargs)
        self._retriever = retriever
        self._priority_keywords = priority_keywords or []

    def get_relevant_documents(self, query: str) -> list[Document]:
        docs = self._retriever.get_relevant_documents(query)

        def score_doc(doc: Document) -> float:
            score = 0
            chapter = doc.metadata.get("chapter", "").lower()
            for kw in self._priority_keywords:
                if kw.lower() in chapter:
                    score += 1
            section = doc.metadata.get("section", "")
            if any(word.lower() in section.lower() for word in query.split()):
                score += 0.5
            return score

        ranked_docs = sorted(docs, key=lambda d: score_doc(d), reverse=True)
        return ranked_docs

# Wrap retriever
priority_keywords = ["labor", "directive principles", "constitution"]
metadata_retriever = MetadataAwareRetriever(retriever, priority_keywords=priority_keywords)

# RetrievalQA wrapper
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=metadata_retriever,
    chain_type="stuff",
    chain_type_kwargs={"prompt": prompt},
    return_source_documents=True,
)

ImportError: cannot import name 'HuggingFaceEmbeddings' from 'langchain.embeddings' (/home/jagannath/my_poc_law/MyPocketLawyer-AI-Powered-Legal-Aid-Assistant/venv/lib/python3.12/site-packages/langchain/embeddings/__init__.py)

In [5]:
# Try a sample query
query = "Who makes plans  on  any  matters  related  to  financial  powers  within  their  respective  jurisdictions."
result = qa_chain({"query": query})

print("\nAnswer:")
print(result["result"])

print("\nSources:")
for doc in result["source_documents"]:
    print(f"- {doc.metadata.get('source')} | Chapter: {doc.metadata.get('chapter')} | Section: {doc.metadata.get('section')}")


---

## New Approach

In [7]:
import os
from pathlib import Path
from dotenv import load_dotenv
import chromadb
from chromadb.config import Settings
from google import genai

# ============================================================
# CONFIG
# ============================================================

load_dotenv()
API_KEY = os.getenv("GEMINI_API_KEY")
if not API_KEY:
    raise ValueError("❌ Missing GEMINI_API_KEY in .env")

client = genai.Client(api_key=API_KEY)

try:
    BASE_DIR = Path(__file__).resolve().parent
except NameError:
    BASE_DIR = Path.cwd()
if BASE_DIR.name == "notebooks":
    BASE_DIR = BASE_DIR.parent

VECTORSTORE_DIR = BASE_DIR / "chroma_db"
COLLECTION_NAME = "legal_docs"

# ============================================================
# EMBEDDING
# ============================================================

def get_query_embedding(text):
    """Generate embedding for the query."""
    response = client.models.embed_content(
        model="models/text-embedding-004",
        contents=[text]
    )
    return response.embeddings[0].values


# ============================================================
# RETRIEVE FUNCTION
# ============================================================

def retrieve_top_k(query, k=5):
    """Retrieve top-k relevant clauses from Chroma."""
    client_chroma = chromadb.PersistentClient(path=str(VECTORSTORE_DIR))

    try:
        collection = client_chroma.get_collection(name=COLLECTION_NAME)
    except Exception:
        raise ValueError(f"❌ Collection '{COLLECTION_NAME}' not found in {VECTORSTORE_DIR}")

    query_emb = get_query_embedding(query)

    results = collection.query(
        query_embeddings=[query_emb],
        n_results=k
    )

    print(f"\n🔍 Query: {query}\n")
    for i, (doc, meta, score) in enumerate(
        zip(results["documents"][0], results["metadatas"][0], results["distances"][0]),
        start=1
    ):
        print(f"--- Result {i} ---")
        print(f"📄 Source: {meta.get('document_title', 'Unknown')}")
        print(f"📘 Part {meta.get('part_number', '')}: {meta.get('part_title', '')}")
        print(f"📜 Article {meta.get('article_number', '')}: {meta.get('article_title', '')}")
        print(f"🔢 Clause {meta.get('clause_index', '')}")
        print(f"📏 Similarity: {score:.4f}")
        print(f"🧾 Text: {doc}\n")


# ============================================================
# TEST SAMPLE QUERY
# ============================================================

if __name__ == "__main__":
    query = "What are the rights related to citizenship in Nepal?"
    retrieve_top_k(query, k=5)


ModuleNotFoundError: No module named 'chromadb'

In [ ]:
!pip install chromadb

  Using cached chromadb-1.3.4-cp39-abi3-macosx_10_12_x86_64.whl.metadata (7.2 kB)
  Using cached build-1.3.0-py3-none-any.whl.metadata (5.6 kB)
  Using cached pybase64-1.4.2-cp313-cp313-macosx_10_13_x86_64.whl.metadata (8.7 kB)
  Using cached uvicorn-0.38.0-py3-none-any.whl.metadata (6.8 kB)
  Using cached posthog-5.4.0-py3-none-any.whl.metadata (5.7 kB)
INFO: pip is looking at multiple versions of chromadb to determine which version is compatible with other requirements. This could take a while.
  Using cached chromadb-1.3.3-cp39-abi3-macosx_10_12_x86_64.whl.metadata (7.2 kB)
  Using cached chromadb-1.3.2-cp39-abi3-macosx_10_12_x86_64.whl.metadata (7.2 kB)
  Using cached chromadb-1.3.0-cp39-abi3-macosx_10_12_x86_64.whl.metadata (7.2 kB)
  Using cached chromadb-1.2.2-cp39-abi3-macosx_10_12_x86_64.whl.metadata (7.2 kB)
  Using cached chromadb-1.2.1-cp39-abi3-macosx_10_12_x86_64.whl.metadata (7.2 kB)
  Using cached chromadb-1.2.0-cp39-abi3-macosx_10_12_x86_64.whl.metadata (7.2 kB)
  Usin